In [13]:
import pyspark

In [14]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder.appName("pysparkproject").getOrCreate()

In [16]:
csvdf = spark.read.csv('C:/Users/ramna/DataScience/DataSet/HouseDataPrediction.csv',header=True)

In [17]:
csvdf.show(5)

+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|           area_type| availability|            location|     size|society|total_sqft|bath|balcony|price|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|Super built-up  Area|       19-Dec|Electronic City P...|    2 BHK|Coomee |      1056|   2|      1|39.07|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|4 Bedroom|Theanmp|      2600|   5|      3|  120|
|      Built-up  Area|Ready To Move|         Uttarahalli|    3 BHK|   null|      1440|   2|      3|   62|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|    3 BHK|Soiewre|      1521|   3|      1|   95|
|Super built-up  Area|Ready To Move|            Kothanur|    2 BHK|   null|      1200|   2|      1|   51|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
only showing top 5 rows



In [18]:
csvdf.count()

13320

In [19]:
csvdf.printSchema()

root
 |-- area_type: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- location: string (nullable = true)
 |-- size: string (nullable = true)
 |-- society: string (nullable = true)
 |-- total_sqft: string (nullable = true)
 |-- bath: string (nullable = true)
 |-- balcony: string (nullable = true)
 |-- price: string (nullable = true)



In [20]:
from pyspark.sql.functions import desc
csvdf.groupby(csvdf.size).count().sort(desc("count")).show(5)

+---------+-----+
|     size|count|
+---------+-----+
|    2 BHK| 5199|
|    3 BHK| 4310|
|4 Bedroom|  826|
|    4 BHK|  591|
|3 Bedroom|  547|
+---------+-----+
only showing top 5 rows



In [21]:
csvdf.select('size').distinct().count()

32

In [22]:
csvdropDF = csvdf.drop('society','balcony','availability')
csvdropDF.show(5)

+--------------------+--------------------+---------+----------+----+-----+
|           area_type|            location|     size|total_sqft|bath|price|
+--------------------+--------------------+---------+----------+----+-----+
|Super built-up  Area|Electronic City P...|    2 BHK|      1056|   2|39.07|
|          Plot  Area|    Chikka Tirupathi|4 Bedroom|      2600|   5|  120|
|      Built-up  Area|         Uttarahalli|    3 BHK|      1440|   2|   62|
|Super built-up  Area|  Lingadheeranahalli|    3 BHK|      1521|   3|   95|
|Super built-up  Area|            Kothanur|    2 BHK|      1200|   2|   51|
+--------------------+--------------------+---------+----------+----+-----+
only showing top 5 rows



In [23]:
csvdropDF.count()

13320

In [24]:
from pyspark.sql.functions import col,split
csvSizeDF = csvdropDF.withColumn('sizeofBhk',split(col("size"),' ').getItem(0))

In [25]:
csvSizeDF.head(5)

[Row(area_type='Super built-up  Area', location='Electronic City Phase II', size='2 BHK', total_sqft='1056', bath='2', price='39.07', sizeofBhk='2'),
 Row(area_type='Plot  Area', location='Chikka Tirupathi', size='4 Bedroom', total_sqft='2600', bath='5', price='120', sizeofBhk='4'),
 Row(area_type='Built-up  Area', location='Uttarahalli', size='3 BHK', total_sqft='1440', bath='2', price='62', sizeofBhk='3'),
 Row(area_type='Super built-up  Area', location='Lingadheeranahalli', size='3 BHK', total_sqft='1521', bath='3', price='95', sizeofBhk='3'),
 Row(area_type='Super built-up  Area', location='Kothanur', size='2 BHK', total_sqft='1200', bath='2', price='51', sizeofBhk='2')]

In [26]:
csvbhkDF = csvSizeDF

In [27]:
csvbhkDF.show(2)

+--------------------+--------------------+---------+----------+----+-----+---------+
|           area_type|            location|     size|total_sqft|bath|price|sizeofBhk|
+--------------------+--------------------+---------+----------+----+-----+---------+
|Super built-up  Area|Electronic City P...|    2 BHK|      1056|   2|39.07|        2|
|          Plot  Area|    Chikka Tirupathi|4 Bedroom|      2600|   5|  120|        4|
+--------------------+--------------------+---------+----------+----+-----+---------+
only showing top 2 rows



In [28]:
from pyspark.sql.functions import concat,lit
csvBhkSizeDF = csvbhkDF.select('area_type','location','size','total_sqft','bath','price',
                               concat('sizeofBhk',lit('BHK')).alias('BhkSize'))

In [29]:
csvBhkSizeDF.show(2)

+--------------------+--------------------+---------+----------+----+-----+-------+
|           area_type|            location|     size|total_sqft|bath|price|BhkSize|
+--------------------+--------------------+---------+----------+----+-----+-------+
|Super built-up  Area|Electronic City P...|    2 BHK|      1056|   2|39.07|   2BHK|
|          Plot  Area|    Chikka Tirupathi|4 Bedroom|      2600|   5|  120|   4BHK|
+--------------------+--------------------+---------+----------+----+-----+-------+
only showing top 2 rows



In [30]:
csvData = csvBhkSizeDF.drop('size')

In [31]:
csvData.show(4)

+--------------------+--------------------+----------+----+-----+-------+
|           area_type|            location|total_sqft|bath|price|BhkSize|
+--------------------+--------------------+----------+----+-----+-------+
|Super built-up  Area|Electronic City P...|      1056|   2|39.07|   2BHK|
|          Plot  Area|    Chikka Tirupathi|      2600|   5|  120|   4BHK|
|      Built-up  Area|         Uttarahalli|      1440|   2|   62|   3BHK|
|Super built-up  Area|  Lingadheeranahalli|      1521|   3|   95|   3BHK|
+--------------------+--------------------+----------+----+-----+-------+
only showing top 4 rows



In [32]:
from pyspark.sql.functions import desc
csvData.groupby(csvData.BhkSize).count().sort(desc("count")).show(5)

+-------+-----+
|BhkSize|count|
+-------+-----+
|   2BHK| 5528|
|   3BHK| 4857|
|   4BHK| 1417|
|   1BHK|  656|
|   5BHK|  356|
+-------+-----+
only showing top 5 rows



In [33]:
csvData.show(5)

+--------------------+--------------------+----------+----+-----+-------+
|           area_type|            location|total_sqft|bath|price|BhkSize|
+--------------------+--------------------+----------+----+-----+-------+
|Super built-up  Area|Electronic City P...|      1056|   2|39.07|   2BHK|
|          Plot  Area|    Chikka Tirupathi|      2600|   5|  120|   4BHK|
|      Built-up  Area|         Uttarahalli|      1440|   2|   62|   3BHK|
|Super built-up  Area|  Lingadheeranahalli|      1521|   3|   95|   3BHK|
|Super built-up  Area|            Kothanur|      1200|   2|   51|   2BHK|
+--------------------+--------------------+----------+----+-----+-------+
only showing top 5 rows



In [34]:
csvData.groupby(csvData.total_sqft).count().sort(desc("count")).show(5)

+----------+-----+
|total_sqft|count|
+----------+-----+
|      1200|  843|
|      1100|  221|
|      1500|  205|
|      2400|  196|
|       600|  180|
+----------+-----+
only showing top 5 rows



In [35]:
csvData.show(5)

+--------------------+--------------------+----------+----+-----+-------+
|           area_type|            location|total_sqft|bath|price|BhkSize|
+--------------------+--------------------+----------+----+-----+-------+
|Super built-up  Area|Electronic City P...|      1056|   2|39.07|   2BHK|
|          Plot  Area|    Chikka Tirupathi|      2600|   5|  120|   4BHK|
|      Built-up  Area|         Uttarahalli|      1440|   2|   62|   3BHK|
|Super built-up  Area|  Lingadheeranahalli|      1521|   3|   95|   3BHK|
|Super built-up  Area|            Kothanur|      1200|   2|   51|   2BHK|
+--------------------+--------------------+----------+----+-----+-------+
only showing top 5 rows



In [40]:
csvSqftDF = csvData

In [41]:
csvSqftDF.show(2)

+--------------------+--------------------+----------+----+-----+-------+
|           area_type|            location|total_sqft|bath|price|BhkSize|
+--------------------+--------------------+----------+----+-----+-------+
|Super built-up  Area|Electronic City P...|      1056|   2|39.07|   2BHK|
|          Plot  Area|    Chikka Tirupathi|      2600|   5|  120|   4BHK|
+--------------------+--------------------+----------+----+-----+-------+
only showing top 2 rows



In [55]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def isSqftfloat(total_sqft):
    sp = total_sqft.split('-')
    if len(sp) == 2:
        return (float(sp[0])+float(sp[1]))/2
    try:
        return float(total_sqft)
    except:
        return None
    
    
csvsqftCleanDF = udf(lambda total_sqft: isSqftfloat(total_sqft), FloatType())

csfSqftHomDF = csvSqftDF.select('area_type','location','BhkSize','bath','price','total_sqft',csvsqftCleanDF('total_sqft')
                  .alias('SqftofHome'))

In [56]:
csfSqftHomDF.show(5)

+--------------------+--------------------+-------+----+-----+----------+----------+
|           area_type|            location|BhkSize|bath|price|total_sqft|SqftofHome|
+--------------------+--------------------+-------+----+-----+----------+----------+
|Super built-up  Area|Electronic City P...|   2BHK|   2|39.07|      1056|    1056.0|
|          Plot  Area|    Chikka Tirupathi|   4BHK|   5|  120|      2600|    2600.0|
|      Built-up  Area|         Uttarahalli|   3BHK|   2|   62|      1440|    1440.0|
|Super built-up  Area|  Lingadheeranahalli|   3BHK|   3|   95|      1521|    1521.0|
|Super built-up  Area|            Kothanur|   2BHK|   2|   51|      1200|    1200.0|
+--------------------+--------------------+-------+----+-----+----------+----------+
only showing top 5 rows



In [57]:
csfSqHomDF = csfSqftHomDF.drop('total_sqft')

In [58]:
csfSqHomDF.show(5)

+--------------------+--------------------+-------+----+-----+----------+
|           area_type|            location|BhkSize|bath|price|SqftofHome|
+--------------------+--------------------+-------+----+-----+----------+
|Super built-up  Area|Electronic City P...|   2BHK|   2|39.07|    1056.0|
|          Plot  Area|    Chikka Tirupathi|   4BHK|   5|  120|    2600.0|
|      Built-up  Area|         Uttarahalli|   3BHK|   2|   62|    1440.0|
|Super built-up  Area|  Lingadheeranahalli|   3BHK|   3|   95|    1521.0|
|Super built-up  Area|            Kothanur|   2BHK|   2|   51|    1200.0|
+--------------------+--------------------+-------+----+-----+----------+
only showing top 5 rows



In [59]:
csfPPSftDF = csfSqHomDF.withColumn('PricePerSqft',csfSqHomDF.price*100000/csfSqHomDF.SqftofHome)

In [60]:
csfPPSftDF.show(3)

+--------------------+--------------------+-------+----+-----+----------+-----------------+
|           area_type|            location|BhkSize|bath|price|SqftofHome|     PricePerSqft|
+--------------------+--------------------+-------+----+-----+----------+-----------------+
|Super built-up  Area|Electronic City P...|   2BHK|   2|39.07|    1056.0|3699.810606060606|
|          Plot  Area|    Chikka Tirupathi|   4BHK|   5|  120|    2600.0|4615.384615384615|
|      Built-up  Area|         Uttarahalli|   3BHK|   2|   62|    1440.0|4305.555555555556|
+--------------------+--------------------+-------+----+-----+----------+-----------------+
only showing top 3 rows



In [61]:
from pyspark.sql.functions import trim
csfPPSftDF.withColumn('location',trim(csfPPSftDF.location))

DataFrame[area_type: string, location: string, BhkSize: string, bath: string, price: string, SqftofHome: float, PricePerSqft: double]

In [62]:
from pyspark.sql.functions import avg
csvLoctDF = csfPPSftDF.groupby("area_type","location","BhkSize","SqftofHome"
                   ,"PricePerSqft","bath","price").agg(avg("SqftofHome").alias('SqftSize'))

In [63]:
csvLoctDF.show(3)

+--------------------+-------------------+-------+----------+------------------+----+-----+--------+
|           area_type|           location|BhkSize|SqftofHome|      PricePerSqft|bath|price|SqftSize|
+--------------------+-------------------+-------+----------+------------------+----+-----+--------+
|          Plot  Area|Devarachikkanahalli|   8BHK|    1350.0|6296.2962962962965|   7|   85|  1350.0|
|Super built-up  Area|        Thanisandra|   2BHK|    1296.0|  6172.83950617284|   2|   80|  1296.0|
|Super built-up  Area| Basaveshwara Nagar|   2BHK|    1200.0| 5416.666666666667|   2|   65|  1200.0|
+--------------------+-------------------+-------+----------+------------------+----+-----+--------+
only showing top 3 rows



In [64]:
csvCleanDF = csvLoctDF

In [65]:
csvCleanDF

DataFrame[area_type: string, location: string, BhkSize: string, SqftofHome: float, PricePerSqft: double, bath: string, price: string, SqftSize: double]

In [66]:
csvColDF = csvCleanDF.drop('SqftofHome')

In [67]:
csvColDF

DataFrame[area_type: string, location: string, BhkSize: string, PricePerSqft: double, bath: string, price: string, SqftSize: double]

In [68]:
csvColCleanDF = csvColDF.withColumn('locationofHome',csvColDF.location)

In [69]:
csvColCleanDF.show(2)

+--------------------+-------------------+-------+------------------+----+-----+--------+-------------------+
|           area_type|           location|BhkSize|      PricePerSqft|bath|price|SqftSize|     locationofHome|
+--------------------+-------------------+-------+------------------+----+-----+--------+-------------------+
|          Plot  Area|Devarachikkanahalli|   8BHK|6296.2962962962965|   7|   85|  1350.0|Devarachikkanahalli|
|Super built-up  Area|        Thanisandra|   2BHK|  6172.83950617284|   2|   80|  1296.0|        Thanisandra|
+--------------------+-------------------+-------+------------------+----+-----+--------+-------------------+
only showing top 2 rows



In [70]:
csvColCleanDF = csvColCleanDF.drop('location')

In [71]:
csvColCleanDF.show(3)

+--------------------+-------+------------------+----+-----+--------+-------------------+
|           area_type|BhkSize|      PricePerSqft|bath|price|SqftSize|     locationofHome|
+--------------------+-------+------------------+----+-----+--------+-------------------+
|          Plot  Area|   8BHK|6296.2962962962965|   7|   85|  1350.0|Devarachikkanahalli|
|Super built-up  Area|   2BHK|  6172.83950617284|   2|   80|  1296.0|        Thanisandra|
|Super built-up  Area|   2BHK| 5416.666666666667|   2|   65|  1200.0| Basaveshwara Nagar|
+--------------------+-------+------------------+----+-----+--------+-------------------+
only showing top 3 rows



In [72]:
csvfeatEngDF = csvColCleanDF.withColumn('AreaType',csvColDF.area_type)

In [73]:
csvfeatEngDF = csvfeatEngDF.drop('area_type')

In [74]:
csvfeatEngDF.show(2)

+-------+------------------+----+-----+--------+-------------------+--------------------+
|BhkSize|      PricePerSqft|bath|price|SqftSize|     locationofHome|            AreaType|
+-------+------------------+----+-----+--------+-------------------+--------------------+
|   8BHK|6296.2962962962965|   7|   85|  1350.0|Devarachikkanahalli|          Plot  Area|
|   2BHK|  6172.83950617284|   2|   80|  1296.0|        Thanisandra|Super built-up  Area|
+-------+------------------+----+-----+--------+-------------------+--------------------+
only showing top 2 rows



In [78]:
csvfeatEngDF.describe(['price']).show(5)

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             12508|
|   mean|115.16789454748971|
| stddev|152.74208540710612|
|    min|                10|
|    max|               995|
+-------+------------------+



In [124]:
csvfeatEngDF.show(3)

+-------+------------------+----+-----+--------+-------------------+--------------------+
|BhkSize|      PricePerSqft|bath|price|SqftSize|     locationofHome|            AreaType|
+-------+------------------+----+-----+--------+-------------------+--------------------+
|   8BHK|6296.2962962962965|   7|   85|  1350.0|Devarachikkanahalli|          Plot  Area|
|   2BHK|  6172.83950617284|   2|   80|  1296.0|        Thanisandra|Super built-up  Area|
|   2BHK| 5416.666666666667|   2|   65|  1200.0| Basaveshwara Nagar|Super built-up  Area|
+-------+------------------+----+-----+--------+-------------------+--------------------+
only showing top 3 rows



In [ ]:
csvfeatEngDF.write.csv('c:/Desktop/tableauModelReport.csv',header=True)

In [125]:
import pandas as pd
df = csvfeatEngDF.toPandas()

In [126]:
df.to_csv('C:/Desktop/bin/tableauModelReport.csv')